In [2]:

from geopandas.tools import sjoin


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json

import fiona
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

#import pydeck as pdk
#import shapefile
import os
import re
import matplotlib 
matplotlib.font_manager._rebuild()
import seaborn as sns

#import mapboxgl
#from mapboxgl.viz import *
#from mapboxgl.utils import create_color_stops

from matplotlib import font_manager, rc
import platform

import requests; from urllib.parse import urlparse


#샾은 현재 설치 하지 않은것들이다ㅠㅠ

In [3]:
acc_cnt = gpd.read_file('3_100by100_xy요약결합.geojson')

In [12]:
acc_cnt_pt = acc_cnt.copy()

In [13]:
acc_cnt_pt.head()

,left,bottom,right,top,xcoord_max,ycoord_max,geometry
0,322800.0,182000.0,322900.0,182100.0,322850.0,182050.0,"MULTIPOLYGON (((322800.000 182000.000, 322800...."
1,322800.0,182300.0,322900.0,182400.0,322850.0,182350.0,"MULTIPOLYGON (((322800.000 182300.000, 322800...."
2,322800.0,182400.0,322900.0,182500.0,322850.0,182450.0,"MULTIPOLYGON (((322800.000 182400.000, 322800...."
3,322900.0,181800.0,323000.0,181900.0,322950.0,181850.0,"MULTIPOLYGON (((322900.000 181800.000, 322900...."
4,322900.0,181900.0,323000.0,182000.0,322950.0,181950.0,"MULTIPOLYGON (((322900.000 181900.000, 322900...."


In [24]:
acc_cnt_pt['geometry_cen'] = acc_cnt_pt['geometry'].centroid  # centroid 공간함수로 중심좌표 반환 

In [25]:
acc_cnt_pt  #어 똑같이 된것 같다!!!!!!!!!!!!!!!!!!!!!!!!!! 기본은 멀티폴리곤 맞는듯

,left,bottom,right,top,x,y,geometry,geometry_cen
0,322800.0,182000.0,322900.0,182100.0,322850.0,182050.0,"MULTIPOLYGON (((322800.000 182000.000, 322800....",POINT (322850.000 182050.000)
1,322800.0,182300.0,322900.0,182400.0,322850.0,182350.0,"MULTIPOLYGON (((322800.000 182300.000, 322800....",POINT (322850.000 182350.000)
2,322800.0,182400.0,322900.0,182500.0,322850.0,182450.0,"MULTIPOLYGON (((322800.000 182400.000, 322800....",POINT (322850.000 182450.000)
3,322900.0,181800.0,323000.0,181900.0,322950.0,181850.0,"MULTIPOLYGON (((322900.000 181800.000, 322900....",POINT (322950.000 181850.000)
4,322900.0,181900.0,323000.0,182000.0,322950.0,181950.0,"MULTIPOLYGON (((322900.000 181900.000, 322900....",POINT (322950.000 181950.000)
...,...,...,...,...,...,...,...,...
77467,368000.0,180800.0,368100.0,180900.0,368050.0,180850.0,"MULTIPOLYGON (((368000.000 180800.000, 368000....",POINT (368050.000 180850.000)
77468,368000.0,180900.0,368100.0,181000.0,368050.0,180950.0,"MULTIPOLYGON (((368000.000 180900.000, 368000....",POINT (368050.000 180950.000)
77469,368000.0,181000.0,368100.0,181100.0,368050.0,181050.0,"MULTIPOLYGON (((368000.000 181000.000, 368000....",POINT (368050.000 181050.000)
77470,368000.0,181100.0,368100.0,181200.0,368050.0,181150.0,"MULTIPOLYGON (((368000.000 181100.000, 368000....",POINT (368050.000 181150.000)


## 주요시설물(카카오 api) 수집

> 카카오api를 활용하여 외부데이터를 수집한다 

~1. 중심좌표 구하기~

(나는 미리 구해놨으니 할 필요 없다만 대전자료는 지오메트리만 줬기때문에 여기서 구했나보다)

(속도 생각하면 이것도 시도해볼만하것 같다 다만 goemetry가 폴리곤이냐 멀티폴리곤이냐 이것도 중요할꺼 같은데;;)

~2. point값을 x,y 좌표계로 변환~

In [19]:
#x,y로 좌표 이름바꾸기
acc_cnt_pt.rename(columns={'xcoord_max':'x','ycoord_max':'y'},inplace=True)  #인덱스는 안해도 그대로 되돌려주나?
acc_cnt_pt.head(1)

,left,bottom,right,top,x,y,geometry
0,322800.0,182000.0,322900.0,182100.0,322850.0,182050.0,"MULTIPOLYGON (((322800.000 182000.000, 322800...."


3. 카카오톡 API 활용

  - 좌표, 반경, 시설물의 keyword, 카테고리, 승인키를 입력하면 중심좌표에서 입력 반경 내에 있는 시설물의 수를 알 수 있도록 하는 함수 정의

In [20]:
def point_to_num_cat(y,x,radius,keyword,category, key):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?&y=" + y + "&x=" + x + "&radius=" + radius + "&query=" + keyword  +"&category_group_code" + category
    result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":key})
    json_obj = result.json()
    global val
    val = json_obj['meta']['pageable_count']
    return val


# key list
# "KakaoAK 4032c69a78d03e95593e9b4d341588f5"
# "KakaoAK 21c25d923a619a70cba3e38b7d594472"   해당코드에 있는 남의것!!

In [22]:
import time
from tqdm import tqdm

### 반경 50M 내 학교의수

In [29]:
ls_scl = []                                                
for i in tqdm(range(0,len(acc_cnt_pt['x']))): 
    ls_scl.append(point_to_num_cat(str(acc_cnt_pt['y'][i]), str(acc_cnt_pt['x'][i]), '50', '학교','SC4',
                                         "KakaoAK 4032c69a78d03e95593e9b4d341588f5"))

100%|██████████████████████████████████████████████████████████████████████████| 77472/77472 [2:04:25<00:00, 10.38it/s]


In [37]:
acc_cnt_pt['school_cnt'].value_counts()

#개판이네;;

45    77472
Name: school_cnt, dtype: int64

In [ ]:
#5/23 기준으로 사용할 곳도 없음